<a href="https://colab.research.google.com/github/Piotr-Klimek/Piotr-Code-Public/blob/main/Seattle_Bachelor_Education_Rates_Compared_to_Obesity_and_Diabetes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U altair

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import altair as alt
import pandas as pd
import numpy as np
print(alt.__version__)


4.2.2


In [3]:
csv_url = 'https://data-seattlecitygis.opendata.arcgis.com/datasets/SeattleCityGIS::racial-and-social-equity-composite-index-current.csv?outSR=%7B%22latestWkid%22%3A2926%2C%22wkid%22%3A2926%7D'

In [4]:
df = pd.read_csv(csv_url)

In [5]:
df

,OBJECTID,GEOID,ACRES_TOTAL,PCT_PEOPLE_OF_COLOR,PCT_ENGLISH_LANG_LEARNERS,PCT_FOREIGN_BORN,PCT_POP_UNDER200PCT_POVERTY,PCT_LESS_BACHELOR_DEGREE,PCT_ADULT_NOLEISUREPHYSACTIV,PCT_ADULT_WITH_DIABETES,...,POP_ADULT_WITH_DISABILITY,POP_ADULT_CIVILIAN_NONINST,COMPOSITE_SCORE,RACE_ELL_ORIGINS_SCORE,SOCIOECON_DISADV_SCORE,HEALTH_DISADV_SCORE,SOCIOECON_DISADV_QUINTILE,PTL_ADULT_WITH_DISABILITY,SHAPE_Length,SHAPE_Area
0,1,53033000402,444.495421,0.3253,0.0501,0.1246,0.1163,0.5104,0.145,0.074,...,872,4342,0.560000,0.40850,0.5735,0.582571,Second Highest,0.960,21397.175298,1.936222e+07
1,2,53033000403,180.931440,0.3699,0.0799,0.1360,0.3653,0.5508,0.174,0.088,...,556,2733,0.821333,0.51800,0.8975,0.880286,Highest Equity Priority/Most Disadvantaged,0.965,14097.026695,7.881186e+06
2,3,53033000700,319.840275,0.4771,0.0890,0.2147,0.1703,0.5147,0.144,0.064,...,481,4536,0.776000,0.70675,0.7270,0.750429,Second Highest,0.619,16134.029915,1.393224e+07
3,4,53033003302,154.943445,0.2672,0.0213,0.1118,0.1599,0.3116,0.089,0.040,...,469,3278,0.327000,0.22550,0.5050,0.333143,Middle,0.829,10550.817927,6.749336e+06
4,5,53033003601,166.572063,0.2835,0.0916,0.1974,0.1873,0.2330,0.095,0.042,...,433,3183,0.446333,0.47850,0.4460,0.412571,Middle,0.789,11894.288895,7.255879e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,176,53033010401,402.741530,0.7678,0.3592,0.4760,0.1783,0.5415,0.209,0.105,...,448,3567,0.772333,0.96975,0.7470,0.489714,Second Highest,0.715,17342.930201,1.754342e+07
176,177,53033011401,304.714627,0.4986,0.0955,0.1969,0.2492,0.6459,0.153,0.063,...,671,4262,0.852000,0.72125,0.8430,0.816286,Highest Equity Priority/Most Disadvantaged,0.857,17478.059159,1.327337e+07
177,178,53033011700,868.765573,0.8409,0.3062,0.3732,0.2719,0.7645,0.226,0.110,...,749,4609,0.902667,0.97100,0.8945,0.675000,Highest Equity Priority/Most Disadvantaged,0.869,29467.760842,3.779155e+07
178,179,53033011901,415.720227,0.7934,0.2538,0.3617,0.1159,0.7084,0.192,0.103,...,455,3516,0.849667,0.95825,0.6215,0.818000,Second Highest,0.750,20474.554777,1.809246e+07


In [6]:
from altair.vegalite.v4.schema.channels import Tooltip

df['PCT_WITH_BACHELOR_DEGREE'] = 1 - df['PCT_LESS_BACHELOR_DEGREE']


In [7]:
from altair.vegalite.v4.schema.channels import Tooltip

diabetes = alt.Chart(df).mark_point().encode(
    x = 'PCT_WITH_BACHELOR_DEGREE',
    y = "PCT_ADULT_WITH_DIABETES",
    tooltip = ["GEOID", "PCT_ADULT_WITH_DIABETES", "PCT_ADULT_WITH_OBESITY", 'PCT_WITH_BACHELOR_DEGREE'],
    #color = alt.Color("PCT_ADULT_WITH_DIABETES", legend = alt.Legend(title = "Diabetes", orient= "bottom"), scale=alt.Scale(scheme = "reds"))  this must be encoded separately for transform_regression to work
  ).properties(
    title= "Rate of Bachelor Degree as compared with Diabetes"
  )

diabetes_color = diabetes.encode(color = alt.Color("PCT_ADULT_WITH_DIABETES", legend = alt.Legend(title = "Diabetes", orient= "bottom"), scale=alt.Scale(scheme = "reds")))

diabetes_fit = diabetes.transform_regression("PCT_WITH_BACHELOR_DEGREE", "PCT_ADULT_WITH_DIABETES").mark_line(color = "green")   #cannot make chart interactive if adding regression line

diabetes_final = diabetes_fit + diabetes_color


obesity = alt.Chart(df).mark_point().encode(
    x = 'PCT_WITH_BACHELOR_DEGREE',
    y = "PCT_ADULT_WITH_OBESITY",
    tooltip = ["GEOID", "PCT_ADULT_WITH_DIABETES", "PCT_ADULT_WITH_OBESITY", 'PCT_WITH_BACHELOR_DEGREE'],
).properties(
     title= "Rate of Bachelor Degree as compared with Obesity"
)

obesity_color = obesity.encode(color = alt.Color("PCT_ADULT_WITH_OBESITY", legend = alt.Legend(title = "Obesity", orient= "bottom"), scale = alt.Scale(scheme="blues")))

obesity_fit= obesity.transform_regression('PCT_WITH_BACHELOR_DEGREE', "PCT_ADULT_WITH_OBESITY").mark_line(color = "green")

obesity_final = obesity_fit + obesity_color

alt.hconcat(diabetes_final, obesity_final
            ).resolve_legend(color='independent'
            ).configure_view(stroke=None).resolve_scale(color = "independent")



alt.HConcatChart(...)

In [8]:
#example where we do not want color gradients to represent percentages

from altair.vegalite.v4.schema.channels import Tooltip

diabetes = alt.Chart(df).mark_point(color = "red").encode(
    x = 'PCT_WITH_BACHELOR_DEGREE',
    y = "PCT_ADULT_WITH_DIABETES",
    tooltip = ["GEOID", "PCT_ADULT_WITH_DIABETES", "PCT_ADULT_WITH_OBESITY", 'PCT_WITH_BACHELOR_DEGREE'],
  ).properties(
    title= "Rate of Bachelor Degree as compared with Diabetes"
  )

diabetes_fit = diabetes.transform_regression("PCT_WITH_BACHELOR_DEGREE", "PCT_ADULT_WITH_DIABETES").mark_line(color = "green")   #cannot make chart interactive if adding regression line
diabetes_final = diabetes + diabetes_fit


obesity = alt.Chart(df).mark_point().encode(
    x = 'PCT_WITH_BACHELOR_DEGREE',
    y = "PCT_ADULT_WITH_OBESITY",
    tooltip = ["GEOID", "PCT_ADULT_WITH_DIABETES", "PCT_ADULT_WITH_OBESITY", 'PCT_WITH_BACHELOR_DEGREE'],
).properties(
     title= "Rate of Bachelor Degree as compared with Obesity In Seattle"
)


obesity_fit = obesity.transform_regression('PCT_WITH_BACHELOR_DEGREE', "PCT_ADULT_WITH_OBESITY").mark_line(color = "green")

obesity_final = obesity_fit + obesity

alt.hconcat(diabetes_final, obesity_final
            ).resolve_legend(color='independent'
            ).resolve_scale(color = "independent")


alt.HConcatChart(...)

In [11]:
import altair as alt
import numpy as np
import pandas as pd


# Perform linear regression using NumPy
coef = np.polyfit(df["PCT_WITH_BACHELOR_DEGREE"], df["PCT_ADULT_WITH_OBESITY"], deg=1)
slope, intercept = coef[0], coef[1]

print("Slope for Obesity:", slope)
print("Y int for Obesity:", intercept)

coef2 = np.polyfit(df["PCT_WITH_BACHELOR_DEGREE"], df["PCT_ADULT_WITH_DIABETES"], deg=1)
slope2, intercept2 = coef2[0], coef2[1]
print("Slope for Diabetes:", slope2)
print("Y int for Diabetes:", intercept2)

#Find the correlation

corr = df['PCT_WITH_BACHELOR_DEGREE'].corr(df['PCT_ADULT_WITH_OBESITY'])
corr2 = df["PCT_WITH_BACHELOR_DEGREE"].corr(df['PCT_ADULT_WITH_DIABETES'])

# Print the correlation coefficient
print('Correlation coefficient for obesity:', corr)
print('Correlation coefficient for diabetes', corr2)

print("Mean Obesity", (df["PCT_ADULT_WITH_OBESITY"].mean()))
print("Mean Diabetes", (df["PCT_ADULT_WITH_DIABETES"].mean()))

Slope for Obesity: -0.08149667058392797
Y int for Obesity: 0.2614335984582198
Slope for Diabetes: -0.10590815354017144
Y int for Diabetes: 0.12977028733367837
Correlation coefficient for obesity: -0.5960963371033043
Correlation coefficient for diabetes -0.7090552364429314
Mean Obesity 0.20821111111111112
Mean Diabetes 0.06060555555555555
